This code combines the cropwise csv files previously generated from the satellite image and drones. It reads four sets of input files as csv - first from the combined GEE data named as combined_crop.csv (eg. combined_mangopolygon.csv), second from the drone data named as crop_Polygon_Drone_Extract.csv (eg. Mango_Polygon_Drone_Extract.csv), third from the Landsat derived VI tiffs named as Crop_Polygon_Landsat_Extract.csv (eg. Mango_Polygon_Landsat_Extract.csv) and the fourth as Crop_Polygon_Sentinel2_Extract.csv (eg. Mango_Polygon_Sentinel2_Extract.csv). As the output, the code will generate three combined csv files by crop which are saved in the "output_folder" folder. The file names are saved as A4I06_fruit_four_combined_polygon_data_for_ML.csv (eg. A4I06_dragonfruit_four_combined_polygon_data_for_ML.csv). All the calculations are done locally and no temporary files are created during the calculations. 

In [85]:
import os
import sys
import subprocess

# This gives the name of the environment directory
print("Environment name:", os.path.basename(sys.prefix))

Environment name: A4I064-ML


In [86]:
# Install necessary packages, if needed

required_packages = ["pandas"]

for package in required_packages:
    try:
        __import__(package if package != "scikit-learn" else "sklearn")
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("All packages have been installed!")

pandas is already installed.
All packages have been installed!


In [87]:
# Import necessary packages
import pandas as pd
import os

In [88]:
# Select one crop at a time
crop_names = ["Rice", "Mango", "Dragonfruit"]

In [89]:
# Input and output folders
input_folder_satellite = r"C:\Users\U8019357\UniSQ\A4I Geospatial Tech - UniSQ Internal - UniSQ Internal\2 - ML\Raw Data\GEE_VIs"
input_folder_landsat = r"C:\Users\U8019357\UniSQ\A4I Geospatial Tech - UniSQ Internal - UniSQ Internal\2 - ML\Raw Data\Landsat_VIs"
input_folder_sentinel2 = r"C:\Users\U8019357\UniSQ\A4I Geospatial Tech - UniSQ Internal - UniSQ Internal\2 - ML\Raw Data\Sentinel2_VIs"
input_folder_drone = r"C:\Users\U8019357\UniSQ\A4I Geospatial Tech - UniSQ Internal - UniSQ Internal\2 - ML\Raw Data\Drone_VIs"

# Output folder to save combined VIs
output_folder = r"C:\Users\U8019357\UniSQ\A4I Geospatial Tech - UniSQ Internal - UniSQ Internal\2 - ML\Processed Data"

In [90]:
# Create the output folder
os.makedirs(output_folder, exist_ok=True)

# Function to find CSV files containing the crop name AND "polygon" in the folder
def find_crop_csv(folder, crop_name):
    files = os.listdir(folder)
    csv_files = [
        f for f in files 
        if f.lower().endswith('.csv') 
           and crop_name.lower() in f.lower() 
           and "polygon" in f.lower()  # only polygon files
           and "minmaxmean" not in f.lower()
    ]
    
    if not csv_files:
        print(f"No CSV file found for crop '{crop_name}' with 'polygon' in {os.path.basename(folder)}")
        return None
    elif len(csv_files) > 1:
        print(f"Multiple CSV files found for crop '{crop_name}' with 'polygon' in {os.path.basename(folder)}:\n {csv_files}\n")
    
    # Return the first matching file
    return os.path.join(folder, csv_files[0])


In [91]:
# Helper function to standardize and parse dates robustly
def process_dates(df, source_name, date_format=None, dayfirst=True, crop_name=None):
    if df.empty:
        print(f"No rows in {source_name} data.")
        return df

    # Standardise 'Date' column name (case/whitespace-insensitive)
    date_col = None
    for col in df.columns:
        if col.strip().lower() == 'date':
            date_col = col
            break
    if date_col is None:
        print(f"No 'Date' column in {source_name} data.")
        return df
    if date_col != 'Date':
        df.rename(columns={date_col: 'Date'}, inplace=True)

    # Work on a clean string view of the Date column
    s = df['Date'].astype(str).str.strip()

    # 1) Remove any time suffix like " 00:00:00", "T00:00:00Z", milliseconds, timezone, etc.
    s_no_time = s.str.replace(r'[T ]\d{2}:\d{2}:\d{2}(\.\d+)?(Z|[+-]\d{2}:\d{2})?$', '', regex=True)

    # 2) Drop trailing ".0" (common if dates came in as floats, e.g., 20240508.0 or 45123.0)
    s_no_time = s_no_time.str.replace(r'\.0$', '', regex=True)

    # Prepare result container
    parsed = pd.Series(pd.NaT, index=df.index, dtype='datetime64[ns]')

    # 3) Handle YYYYMMDD (exactly 8 digits)
    mask_ymd8 = s_no_time.str.fullmatch(r'\d{8}')
    if mask_ymd8.any():
        parsed.loc[mask_ymd8] = pd.to_datetime(s_no_time.loc[mask_ymd8], format='%Y%m%d', errors='coerce')

    # 4) Handle Excel serial dates (5–6 digits; Excel epoch 1899-12-30)
    #    Typical modern serials are ~ 40k–80k. Keep it broad but safe.
    mask_excel = s_no_time.str.fullmatch(r'\d{5,6}')
    if mask_excel.any():
        serial = s_no_time.loc[mask_excel].astype(float).astype('Int64')
        # filter to a sane range to avoid misclassifying short yyyymm-like strings
        ok = serial.between(20000, 90000)
        serial_ok = serial[ok].astype(int)
        parsed.loc[serial_ok.index] = pd.to_datetime('1899-12-30') + pd.to_timedelta(serial_ok, unit='D')

    # 5) Remaining strings → robust mixed parse (or explicit format if provided)
    remaining = parsed.isna()
    if remaining.any():
        try:
            if date_format:
                parsed.loc[remaining] = pd.to_datetime(s_no_time.loc[remaining], format=date_format, errors='coerce')
            else:
                # Pandas 2.0+: infer per element
                parsed.loc[remaining] = pd.to_datetime(s_no_time.loc[remaining], format='mixed', dayfirst=dayfirst, errors='coerce')
        except TypeError:
            # Older pandas without format='mixed'
            parsed.loc[remaining] = pd.to_datetime(s_no_time.loc[remaining], dayfirst=dayfirst, errors='coerce')

    df['Date'] = parsed

    # Report issues (show a few offending raw values to help diagnose)
    missing = df['Date'].isna().sum()
    if missing > 0:
        bad_examples = s.loc[df['Date'].isna()].dropna().unique()[:5]
        print(f"Warning: {missing} invalid 'Date' values in {source_name} for crop {crop_name}. Examples: {bad_examples}")

        # 👇 Add this block here
        bad_rows = df[df['Date'].isna()].copy()
        if not bad_rows.empty:
            print(bad_rows[['Date']].head())
            # Optional: save them for inspection
            # bad_rows.to_csv(os.path.join(output_folder,
            #     f'bad_dates_{source_name}_{crop_name}.csv'), index=False)
    else:
        print(f"All 'Date' values parsed successfully in {source_name} data.")

    return df

In [93]:
# Main loop
for crop_name in crop_names:
    print(f"Processing crop: {crop_name}\n")

    # Find CSV files
    landsat_csv_path = find_crop_csv(input_folder_landsat, crop_name)
    sentinel2_csv_path = find_crop_csv(input_folder_sentinel2, crop_name)
    satellite_csv_path = find_crop_csv(input_folder_satellite, crop_name)
    drone_csv_path = find_crop_csv(input_folder_drone, crop_name)

    # Load DataFrames or empty if missing
    df_satellite = pd.read_csv(satellite_csv_path) if satellite_csv_path else pd.DataFrame()
    df_drone = pd.read_csv(drone_csv_path) if drone_csv_path else pd.DataFrame()
    df_landsat = pd.read_csv(landsat_csv_path) if landsat_csv_path else pd.DataFrame()
    df_sentinel2 = pd.read_csv(sentinel2_csv_path) if sentinel2_csv_path else pd.DataFrame()

    print(f"Rows loaded:\nLandsat: {len(df_landsat)}, Sentinel2: {len(df_sentinel2)}, Satellite: {len(df_satellite)}, Drone: {len(df_drone)}\n")

    # Skip if all empty
    if all(df.empty for df in [df_satellite, df_drone, df_landsat, df_sentinel2]):
        print(f"No data found for {crop_name}. Skipping.\n")
        continue

    # Assign Source column
    if not df_satellite.empty:
        df_satellite['Source'] = df_satellite['Source'].astype(str).apply(lambda x: x if x.endswith('-GEE') else x + '-GEE')
    if not df_drone.empty:
        df_drone['Source'] = 'Drone'
    if not df_landsat.empty:
        df_landsat['Source'] = 'Landsat'
    if not df_sentinel2.empty:
        df_sentinel2['Source'] = 'Sentinel2'

    # Process dates for each DataFrame
    df_landsat = process_dates(df_landsat, 'Landsat', date_format=None, dayfirst=True, crop_name=crop_name)
    df_sentinel2 = process_dates(df_sentinel2, 'Sentinel2', date_format=None, dayfirst=True, crop_name=crop_name)
    df_satellite = process_dates(df_satellite, 'Satellite', date_format=None, dayfirst=True, crop_name=crop_name)
    df_drone = process_dates(df_drone, 'Drone', date_format='%Y%m%d', dayfirst=False, crop_name=crop_name)

    # Combine all DataFrames
    combined_df = pd.concat([df_satellite, df_drone, df_landsat, df_sentinel2], ignore_index=True)

    # Ensure latitude and longitude are numeric
    for col in ['latitude', 'longitude']:
        if col in combined_df.columns:
            combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')

    # Merge ID into Plot_ID: Plot_ID gets priority
    if 'Plot_ID' in combined_df.columns and 'ID' in combined_df.columns:
        combined_df['Plot_ID'] = combined_df['Plot_ID'].combine_first(combined_df['ID'])

    # Drop unwanted ID columns
    for col in ['ID', 'ML_ID']:
        if col in combined_df.columns:
            combined_df.drop(columns=[col], inplace=True)

    # Sort by Plot_ID and Date
    if 'Plot_ID' in combined_df.columns and 'Date' in combined_df.columns:
        combined_df = combined_df.sort_values(by=['Plot_ID', 'Date']).reset_index(drop=True)

    # Save combined DataFrame to CSV
    output_filename = f"A4I064_{crop_name}_Combined_VIs_for_ML.csv"
    output_path = os.path.join(output_folder, output_filename)
    combined_df.to_csv(output_path, index=False)

    print(f"Combined CSV for '{crop_name}' saved to: {os.path.basename(output_path)}")
    print(f"Total rows in combined file for '{crop_name}': {len(combined_df)}\n")

Processing crop: Rice

Multiple CSV files found for crop 'Rice' with 'polygon' in Landsat_VIs:
 ['Rice_Polygon_Landsat_Extract.csv', 'Rice_Polygon_Landsat_Extract_MaxMinMean.csv']

Multiple CSV files found for crop 'Rice' with 'polygon' in Sentinel2_VIs:
 ['Rice_Polygon_Sentinel2_Extract.csv', 'Rice_Polygon_Sentinel2_Extract_maxminmean.csv']

Multiple CSV files found for crop 'Rice' with 'polygon' in GEE_VIs:
 ['VIs_Combined_Rice_Polygon.csv', 'VIs_Landsat_Rice_Polygons.csv', 'VIs_Sentinel2_Rice_Polygons.csv']

Multiple CSV files found for crop 'Rice' with 'polygon' in Drone_VIs:
 ['Rice_Polygon_Drone_Extract.csv', 'Rice_Polygon_Drone_Extract_maxminmean.csv']

Rows loaded:
Landsat: 866, Sentinel2: 720, Satellite: 1400, Drone: 20

All 'Date' values parsed successfully in Landsat data.
All 'Date' values parsed successfully in Sentinel2 data.
All 'Date' values parsed successfully in Satellite data.
All 'Date' values parsed successfully in Drone data.
Combined CSV for 'Rice' saved to: A4I0